In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("QR decomposition.ipynb")

# Matrix Analysis 2024 - EE312

## Week 12 - QR DECOMPOSITION
[LTS2](https://lts2.epfl.ch)

### Objectives
The goal of this week's exercise session is to study some aspects of the QR decomposition and its connections to projections and inverse. We will study two methods that can be used to compute the QR decomposition of a matrix and an application to eigendecomposition.

Please submit your answers **individually**.

Let us consider a square $n\times n$ real matrix $A$ having linearly independent columns (NB: QR factorization is applicable to rectangular matrices, we consider square matrices for simplification). The QR decomposition of $A$ is $A=QR$ where $Q$ is an orthogonal matrix and $R$ and upper-triangular matrix.



# 1. Gram-Schmidt orthogonalization

Reminder:
- The projection operator of a vector $v$ on another $u$ is given by $P_u v = \frac{\langle u,v \rangle}{\langle u,u \rangle}u$.
- Gram-Schmidt orthogonalization process of a basis $(v_1, v_2, ..., v_{n})$ produces an orthormal basis $(e_1, e_2, ..., e_{n})$ as follows:

$u_1 = v_1$, $e_1 = \frac{u_1}{\|u_1\|}$

$u_2 = v_2 - P_{u_1}v_2$, $e_2 = \frac{u_2}{\|u_2\|}$

$u_3 = v_3 - P_{u_2}v_3 - P_{u_1}v_3$, $e_3 = \frac{u_3}{\|u_3\|}$

...

$u_k = v_k - \sum_{j=1}^{k-1} P_{u_j}v_{k}$, $e_k = \frac{u_k}{\|u_k\|}$

<!-- BEGIN QUESTION -->

## 1.1 
We perform the QR decomposition of $A$ by applying Gram-Schmidt to its column vectors of $A$ (denoted by $a_k, k=1,...n)$. $Q$ is made of the column vectors $e_k$, i.e. $Q=(e_1|e_2|...|e_{n})$. Let us denote by $r_{jk}$ the coefficients of $R$.

Prove that $r_{kk} = \|u_k\|$, $r_{jk} = <e_j, a_k>$ for $j<k$ and $r_{jk} = 0$ for $j>k$.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 1.2
Implement a function that performs the QR decomposition of a square $n\times n$ matrix using the Gram-Schmidt orthogonalization process

In [ ]:
import numpy as np

In [ ]:
def qr_decomp_gs(A):
    """
    Performs the QR decomposition of the matrix using Gram-Schmidt
    
    Parameters: 
      - A: input matrix nxn
      
    Returns:
      - Q matrix (orthogonal)
      - R matrix (upper triangular)
    """

    n = A.shape[0]
    Q = np.zeros((n,n))
    R = np.zeros((n,n))
    ...

In [ ]:
grader.check("q1p2")

## 1.3
Using the properties of Q, write a routine that can solve a linear system $Ax=b$ using the QR factorization of $A$ ($A$ being a $n\times n$ real matrix).
What property do the coefficients of $R$ need to satisfy for the solution to exist ?

In [ ]:
def solve_gs(A, b):
    """
    Solve a linear system Ax=b using QR decomposition
    
    Parameters:
      - A input matrix nxn
      - b result vector (length n)
    
    Returns:
      - solution vector x (length n)
    """
    n = A.shape[0]
    x = np.zeros(n)
    ...

In [ ]:
# Small numerical example
A=np.array([[12,-51,4],[6,167,-68],[-4,24,-41]])
b=np.array([1, -1, 1])

In [ ]:
x = solve_gs(A, b)

In [ ]:
A@x

In [ ]:
grader.check("q1p3")

<!-- BEGIN QUESTION -->

## 1.4
Consider the matrix 
$B=\begin{pmatrix}1 & 1 & 1 \\ \varepsilon & 0 & 0\\ 0 & \varepsilon & 0\end{pmatrix}$, with $\varepsilon$ being small enough (typically $10^{-8}$ or smaller). In that case, is the method used to compute Q and R still reliable (and why) ?

In [ ]:
eps=1e-8

_Type your answer here, replacing this text._

In [ ]:
B = ...

In [ ]:
# Check the orthogonality of Q
...

In [ ]:
# Check the validity of the decomposition
...

<!-- END QUESTION -->

# 2. Householder reflections

We will now study and implement another method to perform the QR decomposition that is more resistant to the numerical issues mentioned above.

<!-- BEGIN QUESTION -->

## 2.1 
Let $v$ be a unit vector and the associated transform $H_v=I-2vv^T$. Prove that $H_v$ is orthogonal and that it preserves the norm. What is the effect of $H_v$ on a vector $u$ orthogonal to $v$ ?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 2.2 
What is the effect of $H_v$ on any vector $u$ (drawing the planar case can be of help).

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 2.3
The goal is now to design $n$ reflection operators $H_{v_1}, H_{v_2}, ..., H_{v_n}$ s.t. we have
$$
H_{v_n}...H_{v_2}H_{v_1}A = R,
$$
where $R$ is an upper triangular matrix. 

The reflection operations are meant to be "progressive", i.e. $H_{v_k}...H_{v_1}A$ has its $k$ first columns upper triangular.

- Find $v_1$ and $\alpha$ s.t. $H_{v_1}a_1 = \alpha e_1$, where $a_1$ is the first column of $A$ and $e_1 = \begin{pmatrix}1\\0\\ \vdots \\0 \end{pmatrix}$.

- Let us now generalize the previous step to find $H_{v_k}$. Given a vector $x = \begin{pmatrix}x^U \\ x^L\end{pmatrix}$ with $x^U \in\mathbb{R}^k$
and $x^L\in\mathbb{R}^{n-k}$. 

Find a vector $v_k$ s.t. $H_{v_k}x = \begin{pmatrix}x^U \\ \alpha e_1^L \end{pmatrix}$, where $\alpha\in\mathbb{R}$ and 
$e_1^L = \begin{pmatrix}1\\0\\ \vdots \\0 \end{pmatrix} \in \mathbb{R}^{n-k}$. 

It might be of help to write $v_k=\begin{pmatrix}v_k^U\\v_k^L\end{pmatrix}$, with $v_k^U \in\mathbb{R}^k$
and $v_k^L\in\mathbb{R}^{n-k}$. 

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## 2.4
We now have all we need to compute the $H_{v_k}$ matrices. Implement the QR decomposition using the above results. 
You should have found that the sign of $\alpha$ can be either positive or negative. Choose $\alpha$ to have the sign opposite to the one of $x^L_k[0]$. (Use the `numpy.sign`function).

Try your implementation on the $B$ matrix seen in ex. 1. What is the advantage of this implementation ?

In [ ]:
def qr_decomp_hh(A):
    """
    Performs the QR decomposition of the matrix using Householder reflections
    
    Parameters: 
      - A: input matrix nxn
      
    Returns:
      - Q matrix (orthogonal)
      - R matrix (upper triangular)
    """
    n = A.shape[0]
    ...

In [ ]:
qh, rh = qr_decomp_hh(B)

In [ ]:
# Check orthogonality of Q
...

In [ ]:
# Check validity of solution
...

# 3. Eigendecomposition and QR

We will now study some aspects of how to actually perform (numerically) the eigendecomposition of a matrix. 

In the following we will suppose that $A$ has distinct eigenvalues, and we will write them as $\lambda_1, ..., \lambda_n$ s.t. $|\lambda_1| > |\lambda_2|...>|\lambda_n|$, and denote their associated normalized eigenvectors as $v_1, v_2..., v_n$.

<!-- BEGIN QUESTION -->

## 3.1 Power method

Let us consider a vector $w\in\mathbb{R}^n$ of norm 1, and its associated eigendecomposition $w = \sum_{k=1}^n\alpha_k v_k$. 

Compute $A^pw$ (for $p\in\mathbb{N}$, $k>0$). Assuming that $\alpha_1\neq 0$ for the chosen $w$, use this result to find an algorithm to compute $\lambda_1$ and $v_1$ (you may want to introduce another vector $x\in\mathbb{R}^n$ s.t. $x^Tv_1\neq 0$ and consider the quantity $x^TA^pw$) ?

How would you proceed for $\lambda_2$ and $v_2$ and the other eigenvalues ?

What are the limitations of this method ?

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 3.2 QR method

### 3.2.1
Prove that the matrix $B_1=(I-v_1v_1^T)A$ has eigenvalues $0, \lambda_2, ..., \lambda_n$, with corresponding eigenvectors $v_1, (I-v_1v_1^T)v_2, ..., (I-v_1v_1^T)v_n$. How about the matrices $B_2=(I-v_2v_2^T)(I-v_1v_1^T)A, B_3=(I-v_3v_3^T)(I-v_2v_2^T)(I-v_1v_1^T)A$, etc. when supposing that $v_1, v_2...$ are orthogonal ?

Using question 1 (you can show the operator $I-v^Tv$ is indeed the basis operation of Gram-Schmidt), reformulate the following algorithm using the QR decomposition of $AV^{(k)}$ (denoted by $V^{(k+1)}R^{(k+1)}$):
- Start with $v_1^{(0)}, v_2^{(0)}, ..., v_n^{(0)}=V^{(0)}$
- Iterate for $k=0, 1, ... $
  - $v_1^{(k+1)}=Av_1^{(k)}$ and normalize $v_1^{(k+1)}$
  - $v_2^{(k+1)} = (I-v_1^{(k+1)}v_1^{(k+1)T})Av_1^{(k)}$ and normalize $v_2^{(k+1)}$
  - ...
  - $v_n^{(k+1)} = (I-v_{n-1}^{(k+1)}v_{n-1}^{(k+1)T})...(I-v_1^{(k+1)}v_1^{(k+1)T})Av_1^{(k)}$ and normalize $v_n^{(k+1)}$
  


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 3.2.2

How does the algorithm of question 3.2.1 relates to the one from question 3.1 ?


_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 3.2.3

Let us introduce the matrix $A^{(k)}=V^{(k)T}AV^{(k)}$, where $(v_1^{(k)}, v_2^{(k)}, ..., v_n^{(k)})=V^{(k)}$. Let us also introduce the QR decomposition of $A^{(k)}=Q^{(k+1)}R^{(k+1)}$.

Prove that $A^{(k+1)} = R^{(k+1)}Q^{(k+1)}$. What is the interest of this sequence of matrices $A^{(k)}$ regarding the eigenvalues of $A$ ?

Hint: remember the QR decomposition is unique.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

### 3.2.4

Implement the algorithm that generates the sequence $A^{(k)}$. The diagonal elements of $A^{(k)}$ will converge towards the eigenvalues of $A$ (you do not need to prove this result).

In [ ]:
def eigendecomposition_qr(A, num_iter=20):
    """
    Computes the eigenvalues of A using the QR iteration method
    
    Parameters:
      - A: input nxn matrix
      - num_iter: number of QR iterations to perform
      
    Returns:
      - a vector containing the eigenvalues
    """
    Ak = A
    ...

In [ ]:
grader.check("q3p24")

Despite the assumption about disctinct eigenvalues, you can verify the eigenvalues of the matrix studied in week 7 are computed correctly:

In [ ]:
B = np.array([[1, 0.25, 0], [0, 0.5, 0], [0, 0.25, 1]])

In [ ]:
eigendecomposition_qr(B, 150)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload your notebook and separate pdf for theoretical questions if needed

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)